In [1]:
import pystac_client
import xarray as xr
import matplotlib.pyplot as plt
import numpy as np
from itertools import compress
import pandas as pd

In [2]:
# opening STAC catalog

catalog = pystac_client.Client.open(
    #"https://storage.googleapis.com/dgds-data-public/coclico/coclico-stac/catalog.json" # cloud API
    "https://storage.googleapis.com/dgds-data-public/gca/SOTC/gca-stac_sotc/catalog.json" # P drive API 
)
#catalog

c:\Users\rowe\AppData\Local\miniconda3\envs\coclico\Lib\site-packages\pystac_client\client.py:186: NoConformsTo: Server does not advertise any conformance classes.
  warnings.warn(NoConformsTo())


In [3]:
list(catalog.get_children())

[<CollectionClient id=shore_mon_fut>,
 <CollectionClient id=sed_class>,
 <CollectionClient id=shore_mon>,
 <CollectionClient id=world_pop>,
 <CollectionClient id=world_gdp>,
 <CollectionClient id=shore_mon_hr>,
 <CollectionClient id=shore_mon_drivers>,
 <CollectionClient id=esl_gwl>,
 <CollectionClient id=sub_threat>,
 <CollectionClient id=Exp_world_pop>,
 <CollectionClient id=Exp_world_pop_parquet>,
 <CollectionClient id=Haz-ESL>,
 <CollectionClient id=Vul-Inf_Mort>,
 <CollectionClient id=Haz-GCM_90-100>,
 <CollectionClient id=Exp-NS_Slopes>,
 <CollectionClient id=Vul_MYS>,
 <CollectionClient id=Vul_RS>,
 <CollectionClient id=Haz-GCM_40-50>,
 <CollectionClient id=Haz-ERA5_04-14>,
 <CollectionClient id=Vul-GDP_Cap>,
 <CollectionClient id=Haz-GCM_04-14>,
 <CollectionClient id=Haz-Land_Sub_2010_COGs>,
 <CollectionClient id=Haz-Land_Sub_2040_COGs>,
 <CollectionClient id=Vul-Roads_Density_COGs>,
 <CollectionClient id=Exp-Delta_DEM_COGs>]

In [4]:
href = catalog.get_child("sed_class").assets["data"].href
ds = xr.open_zarr(href)
ds

<xarray.Dataset>
Dimensions:         (stations: 1826995)
Coordinates:
    continent       (stations) |S23 dask.array<chunksize=(28547,), meta=np.ndarray>
    country         (stations) |S40 dask.array<chunksize=(28547,), meta=np.ndarray>
    lat             (stations) float64 dask.array<chunksize=(57094,), meta=np.ndarray>
    lon             (stations) float64 dask.array<chunksize=(57094,), meta=np.ndarray>
    transect_geom   (stations) |S93 dask.array<chunksize=(14274,), meta=np.ndarray>
    transect_id     (stations) |S15 dask.array<chunksize=(57094,), meta=np.ndarray>
Dimensions without coordinates: stations
Data variables:
    sediment_label  (stations) int32 dask.array<chunksize=(114188,), meta=np.ndarray>
Attributes: (12/21)
    AUTHOR:              Hulskamp, R., Lhermitte, S., Aarninkhof, S., Luijend...
    CITATION:            Hulskamp, R. et al. (2021). Global distribution of m...
    COMMENT:             Classification label: {0: sand, 1: mud, 2: coastal c...
    CRS:                 EPSG:4326
    Conventions:         CF-1.8
    DESCRIPTION:         The Beach Sediment Classification dataset indicates ...
    ...                  ...
    SPATIAL_EXTENT:      [-180, -90, 180, 90]
    TAGS:                ['civil engineering', 'geomorphology', 'remote sensi...
    TEMPORAL_EXTENT:     []
    TITLE:               Beach Sediment Classification
    TITLE_ABBREVIATION:  sed_class
    UNITS:

In [12]:
df=ds.to_dataframe()
df

,continent,country,lat,lon,sediment_label,transect_geom,transect_id
stations,,,,,,,
0,b'South America',b'Chile',-54.459554,-72.982524,2,b'LINESTRING (-72.98252423386046 -54.459553896...,b'BOX_028_000_0'
1,b'South America',b'Chile',-54.460984,-72.986282,2,b'LINESTRING (-72.98628220253276 -54.460983853...,b'BOX_028_000_1'
2,b'South America',b'Chile',-54.462808,-72.993247,2,b'LINESTRING (-72.99324679465136 -54.462807671...,b'BOX_028_000_2'
3,b'South America',b'Chile',-54.463640,-72.999769,2,b'LINESTRING (-72.99976896352308 -54.463640451...,b'BOX_028_000_3'
4,b'South America',b'Chile',-54.463962,-73.005142,2,b'LINESTRING (-73.00514240209199 -54.463962458...,b'BOX_028_000_4'
...,...,...,...,...,...,...,...
1826990,b'Europe',b'Russia',65.622118,39.890997,3,b'LINESTRING (39.8909965590071 65.622118191113...,b'BOX_211_092_25'
1826991,b'Europe',b'Russia',65.621258,39.888420,2,b'LINESTRING (39.8884197797104 65.621258185258...,b'BOX_211_092_26'
1826992,b'Europe',b'Russia',65.619990,39.885143,2,b'LINESTRING (39.88514262410951 65.61999031540...,b'BOX_211_092_27'


In [13]:
label_dict = {0: 'sand', 1 : 'mud', 2: 'cliff', 3: 'vegetated', 4: 'other'}

df["sediment_label"] = df["sediment_label"].map(label_dict)
df

,continent,country,lat,lon,sediment_label,transect_geom,transect_id
stations,,,,,,,
0,b'South America',b'Chile',-54.459554,-72.982524,cliff,b'LINESTRING (-72.98252423386046 -54.459553896...,b'BOX_028_000_0'
1,b'South America',b'Chile',-54.460984,-72.986282,cliff,b'LINESTRING (-72.98628220253276 -54.460983853...,b'BOX_028_000_1'
2,b'South America',b'Chile',-54.462808,-72.993247,cliff,b'LINESTRING (-72.99324679465136 -54.462807671...,b'BOX_028_000_2'
3,b'South America',b'Chile',-54.463640,-72.999769,cliff,b'LINESTRING (-72.99976896352308 -54.463640451...,b'BOX_028_000_3'
4,b'South America',b'Chile',-54.463962,-73.005142,cliff,b'LINESTRING (-73.00514240209199 -54.463962458...,b'BOX_028_000_4'
...,...,...,...,...,...,...,...
1826990,b'Europe',b'Russia',65.622118,39.890997,vegetated,b'LINESTRING (39.8909965590071 65.622118191113...,b'BOX_211_092_25'
1826991,b'Europe',b'Russia',65.621258,39.888420,cliff,b'LINESTRING (39.8884197797104 65.621258185258...,b'BOX_211_092_26'
1826992,b'Europe',b'Russia',65.619990,39.885143,cliff,b'LINESTRING (39.88514262410951 65.61999031540...,b'BOX_211_092_27'


In [14]:

Exposure_level = {'cliff': 1 , 'vegetated': 2 , 'other': 3, 'sand': 4, 'mud': 5}

df["Exposure_level"] = df["sediment_label"].map(Exposure_level)
df

,continent,country,lat,lon,sediment_label,transect_geom,transect_id,Exposure_level
stations,,,,,,,,
0,b'South America',b'Chile',-54.459554,-72.982524,cliff,b'LINESTRING (-72.98252423386046 -54.459553896...,b'BOX_028_000_0',1
1,b'South America',b'Chile',-54.460984,-72.986282,cliff,b'LINESTRING (-72.98628220253276 -54.460983853...,b'BOX_028_000_1',1
2,b'South America',b'Chile',-54.462808,-72.993247,cliff,b'LINESTRING (-72.99324679465136 -54.462807671...,b'BOX_028_000_2',1
3,b'South America',b'Chile',-54.463640,-72.999769,cliff,b'LINESTRING (-72.99976896352308 -54.463640451...,b'BOX_028_000_3',1
4,b'South America',b'Chile',-54.463962,-73.005142,cliff,b'LINESTRING (-73.00514240209199 -54.463962458...,b'BOX_028_000_4',1
...,...,...,...,...,...,...,...,...
1826990,b'Europe',b'Russia',65.622118,39.890997,vegetated,b'LINESTRING (39.8909965590071 65.622118191113...,b'BOX_211_092_25',2
1826991,b'Europe',b'Russia',65.621258,39.888420,cliff,b'LINESTRING (39.8884197797104 65.621258185258...,b'BOX_211_092_26',1
1826992,b'Europe',b'Russia',65.619990,39.885143,cliff,b'LINESTRING (39.88514262410951 65.61999031540...,b'BOX_211_092_27',1


In [15]:
df.to_csv(r"P:\11209197-018-global-coastal-atlas\MSc_students\ClenmarRowe\Data\Processed_data\02_Exposure\Sediment_type\Global_Sed_type.csv",index=False)

In [16]:
import geopandas as gpd
df=pd.read_csv(r"P:\11209197-018-global-coastal-atlas\MSc_students\ClenmarRowe\Data\Processed_data\02_Exposure\Sediment_type\Global_Sed_type.csv")
data_points = gpd.GeoDataFrame(
    df, geometry=gpd.points_from_xy(df.lon, df.lat), crs="EPSG:4326"
)
data_points

,continent,country,lat,lon,sediment_label,transect_geom,transect_id,Exposure_level,geometry
0,b'South America',b'Chile',-54.459554,-72.982524,cliff,b'LINESTRING (-72.98252423386046 -54.459553896...,b'BOX_028_000_0',1,POINT (-72.98252 -54.45955)
1,b'South America',b'Chile',-54.460984,-72.986282,cliff,b'LINESTRING (-72.98628220253276 -54.460983853...,b'BOX_028_000_1',1,POINT (-72.98628 -54.46098)
2,b'South America',b'Chile',-54.462808,-72.993247,cliff,b'LINESTRING (-72.99324679465136 -54.462807671...,b'BOX_028_000_2',1,POINT (-72.99325 -54.46281)
3,b'South America',b'Chile',-54.463640,-72.999769,cliff,b'LINESTRING (-72.99976896352308 -54.463640451...,b'BOX_028_000_3',1,POINT (-72.99977 -54.46364)
4,b'South America',b'Chile',-54.463962,-73.005142,cliff,b'LINESTRING (-73.00514240209199 -54.463962458...,b'BOX_028_000_4',1,POINT (-73.00514 -54.46396)
...,...,...,...,...,...,...,...,...,...
1826990,b'Europe',b'Russia',65.622118,39.890997,vegetated,b'LINESTRING (39.8909965590071 65.622118191113...,b'BOX_211_092_25',2,POINT (39.89100 65.62212)
1826991,b'Europe',b'Russia',65.621258,39.888420,cliff,b'LINESTRING (39.8884197797104 65.621258185258...,b'BOX_211_092_26',1,POINT (39.88842 65.62126)
1826992,b'Europe',b'Russia',65.619990,39.885143,cliff,b'LINESTRING (39.88514262410951 65.61999031540...,b'BOX_211_092_27',1,POINT (39.88514 65.61999)
1826993,b'Europe',b'Russia',65.618722,39.881865,sand,b'LINESTRING (39.88186539356586 65.61872241646...,b'BOX_211_092_28',4,POINT (39.88187 65.61872)
